# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [1]:
import requests
import pyowm
import json

api_key = '7d6163678299ac4e90ed1434d81ca873'

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [3]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    '7d6163678299ac4e90ed1434d81ca873' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'cod': 401,
 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [ ]:
import requests
import json

api_key = "7d6163678299ac4e90ed1434d81ca873"

params = {
    "appid": api_key,
    "q": "London",      
    "units": "metric"   
}

url = "https://api.openweathermap.org/data/2.5/weather"

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=4))  
else:
    print(f"Error {response.status_code}: {response.json().get('message', 'Unknown error')}")

{
    "coord": {
        "lon": -0.1257,
        "lat": 51.5085
    },
    "weather": [
        {
            "id": 800,
            "main": "Clear",
            "description": "clear sky",
            "icon": "01d"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 9.86,
        "feels_like": 8.29,
        "temp_min": 8.49,
        "temp_max": 11.12,
        "pressure": 1016,
        "humidity": 70,
        "sea_level": 1016,
        "grnd_level": 1012
    },
    "visibility": 10000,
    "wind": {
        "speed": 3.09,
        "deg": 260
    },
    "clouds": {
        "all": 4
    },
    "dt": 1732632766,
    "sys": {
        "type": 2,
        "id": 2075535,
        "country": "GB",
        "sunrise": 1732606632,
        "sunset": 1732636740
    },
    "timezone": 0,
    "id": 2643743,
    "name": "London",
    "cod": 200
}


## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [5]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 9.85°C, Humidity: 70%, Wind Speed: 3.09 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [ ]:
from pyowm.owm import OWM

owm = OWM(api_key)
weather_mgr = owm.weather_manager()

observation = weather_mgr.weather_at_place("London")
weather = observation.weather

temperature = weather.temperature('celsius')['temp']
humidity = weather.humidity
wind_speed = weather.wind()['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 9.87°C, Humidity: 70%, Wind Speed: 3.09 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [7]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [8]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2024-11-26 18:00:00, Temp: 9.21°C
Time: 2024-11-26 21:00:00, Temp: 8.6°C
Time: 2024-11-27 00:00:00, Temp: 7.39°C
Time: 2024-11-27 03:00:00, Temp: 7.75°C
Time: 2024-11-27 06:00:00, Temp: 9.15°C
Time: 2024-11-27 09:00:00, Temp: 5.61°C
Time: 2024-11-27 12:00:00, Temp: 6.5°C
Time: 2024-11-27 15:00:00, Temp: 6.38°C
Time: 2024-11-27 18:00:00, Temp: 5.35°C
Time: 2024-11-27 21:00:00, Temp: 4.44°C
Time: 2024-11-28 00:00:00, Temp: 3.83°C
Time: 2024-11-28 03:00:00, Temp: 3.49°C
Time: 2024-11-28 06:00:00, Temp: 3.2°C
Time: 2024-11-28 09:00:00, Temp: 3.66°C
Time: 2024-11-28 12:00:00, Temp: 6.22°C
Time: 2024-11-28 15:00:00, Temp: 6.21°C
Time: 2024-11-28 18:00:00, Temp: 4.76°C
Time: 2024-11-28 21:00:00, Temp: 4.54°C
Time: 2024-11-29 00:00:00, Temp: 4.76°C
Time: 2024-11-29 03:00:00, Temp: 4.9°C
Time: 2024-11-29 06:00:00, Temp: 5.13°C
Time: 2024-11-29 09:00:00, Temp: 6.07°C
Time: 2024-11-29 12:00:00, Temp: 8.2°C
Time: 2024-11-29 15:00:00, Temp: 7.99°C
Time: 2024-11-29 18:00:00, Temp: 7.23°C
Time:

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [ ]:
from pyowm.owm import OWM

api_key = "7d6163678299ac4e90ed1434d81ca873"

owm = OWM(api_key)
mgr = owm.weather_manager()

forecast = mgr.forecast_at_place("New York", "3h").forecast

for weather in forecast:
    print(f"Time: {weather.reference_time('iso')}, Temp: {weather.temperature('celsius')['temp']}°C")

Time: 2024-11-26 15:00:00+00:00, Temp: 11.74°C
Time: 2024-11-26 18:00:00+00:00, Temp: 12.16°C
Time: 2024-11-26 21:00:00+00:00, Temp: 11.99°C
Time: 2024-11-27 00:00:00+00:00, Temp: 9.02°C
Time: 2024-11-27 03:00:00+00:00, Temp: 7.72°C
Time: 2024-11-27 06:00:00+00:00, Temp: 6.64°C
Time: 2024-11-27 09:00:00+00:00, Temp: 5.85°C
Time: 2024-11-27 12:00:00+00:00, Temp: 5.14°C
Time: 2024-11-27 15:00:00+00:00, Temp: 6.91°C
Time: 2024-11-27 18:00:00+00:00, Temp: 9.11°C
Time: 2024-11-27 21:00:00+00:00, Temp: 9.51°C
Time: 2024-11-28 00:00:00+00:00, Temp: 8.97°C
Time: 2024-11-28 03:00:00+00:00, Temp: 8.9°C
Time: 2024-11-28 06:00:00+00:00, Temp: 8.86°C
Time: 2024-11-28 09:00:00+00:00, Temp: 8.54°C
Time: 2024-11-28 12:00:00+00:00, Temp: 7.14°C
Time: 2024-11-28 15:00:00+00:00, Temp: 7.54°C
Time: 2024-11-28 18:00:00+00:00, Temp: 8.08°C
Time: 2024-11-28 21:00:00+00:00, Temp: 7.18°C
Time: 2024-11-29 00:00:00+00:00, Temp: 6.46°C
Time: 2024-11-29 03:00:00+00:00, Temp: 6.05°C
Time: 2024-11-29 06:00:00+00:00,

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [10]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 283.01°C, Fahrenheit: 541.418°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [ ]:
from pyowm.owm import OWM

owm = OWM(api_key)
mgr = owm.weather_manager()

observation = mgr.weather_at_place("London")
weather = observation.weather

temperature_celsius = weather.temperature('celsius')['temp']
temperature_fahrenheit = weather.temperature('fahrenheit')['temp']

print(f"Temperature in Celsius: {temperature_celsius}°C")
print(f"Temperature in Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 9.79°C
Temperature in Fahrenheit: 49.62°F
